In [1]:
import sys

In [2]:
import scvi

/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import scanpy as sc

In [ ]:
import sys
import scvi
import scanpy as sc
from inVAE import FinVAE
import scvi
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import Classifier
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.metrics import classification_report
from scarches.models.scpoli import scPoli

import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

## Classification Results

In [ ]:
# Specify the directory path
directory = '/home/icb/ferdinand.kapl/project-inpossible/outputs/transfer_labeling'

# Initialize an empty DataFrame to store the concatenated data
concatenated_df = pd.DataFrame()

# Iterate over each file in the directory
for model_name in ['invae', 'scanvi']:
    for filename in os.listdir(os.path.join(directory, model_name)):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, model_name, filename)
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            df['file_path'] = file_path
            # Concatenate the DataFrame to the existing data
            concatenated_df = pd.concat([concatenated_df, df])

# Print the concatenated DataFrame
print(concatenated_df)

## Visualize embeddings

In [2]:
adata = sc.read_h5ad('data/heart_transfer_labeling.h5ad')
adata.var_names_make_unique()

/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
adata.obs['partition'].value_counts()

train    167633
test      84801
val       18625
Name: partition, dtype: int64

In [5]:
adata.layers['counts'] = adata.layers['counts'].astype(np.float32)

In [6]:
adata_train_val = adata[adata.obs['partition'] != 'test']

adata_train = adata[adata.obs['partition'] == 'train']
adata_val = adata[adata.obs['partition'] == 'val']
adata_test = adata[adata.obs['partition'] == 'test']

### Visualize inVAE emb

In [ ]:
checkpoint_path = 'checkpoints/invae/model_sep_lin_class_invae_cell_type_1720543205.pt'

In [ ]:
# Load Model
model = FinVAE.load_model(save_path=checkpoint_path, adata=adata_train_val)

Initializing the model and loading weights from given checkpoint...
Data loading done!


In [10]:
adata.obsm['X_FinVAE'] = model.get_latent_representation(adata)

Calculating latent representation of passed adata by trying to transfer setup from the adata the model was trained on!


In [11]:
sc.pp.neighbors(adata, use_rep='X_FinVAE')
sc.tl.umap(adata)

In [12]:
adata.obsm['X_FinVAE_umap'] = adata.obsm['X_umap']

In [13]:
sc.set_figure_params(dpi=200,figsize=(6,4),fontsize=10,frameon=False)

In [ ]:
use_emb = 'X_FinVAE_umap'

In [ ]:
sc.pl.embedding(adata, basis=use_emb, color='predicted_labels')

In [ ]:
sc.pl.embedding(adata, basis=use_emb, color='partition')

In [ ]:
sc.pl.embedding(adata, basis=use_emb, color='condition')

In [ ]:
sc.pl.embedding(adata, basis=use_emb, color='publication')

### Visualize scANVI emb

In [19]:
adata_train_val = adata_train_val.copy()

In [ ]:
def _eval_model(model, model_name, classifier = None):
    # Eval model from args: depending on model
    # linear, knn, ...
    if classifier is not None:
        pred_train = classifier.predict(model.get_latent_representation(adata_train))
        pred_val = classifier.predict(model.get_latent_representation(adata_val))
        pred_test = classifier.predict(model.get_latent_representation(adata_test))
    elif model_name == 'inVAE':
        pred_train = model.predict(adata_train, dataset_type='train')
        pred_val = model.predict(adata_val, dataset_type='val')
        pred_test = model.predict(adata_test, dataset_type='test')
    elif model_name == 'scANVI':
        pred_train = model.predict(adata_train)
        pred_val = model.predict(adata_val)
        pred_test = model.predict(adata_test)
    else:
        print('No prediction method implemented for model', model_name)

    label_key = 'predicted_labels'
    # Calculate metrics
    y_train = adata_train.obs[label_key]
    y_val = adata_val.obs[label_key]
    y_test = adata_test.obs[label_key]

    # Acc
    train_acc = (pred_train == y_train).mean()
    val_acc = (pred_val == y_val).mean()
    test_acc = (pred_test == y_test).mean()

    # F1
    # Calculate the weighted F1 score
    weighted_f1_train = f1_score(y_train, pred_train, average='weighted')
    weighted_f1_val = f1_score(y_val, pred_val, average='weighted')
    weighted_f1_test = f1_score(y_test, pred_test, average='weighted')

    # Calculate the macro F1 score
    macro_f1_train = f1_score(y_train, pred_train, average='macro')
    macro_f1_val = f1_score(y_val, pred_val, average='macro')
    macro_f1_test = f1_score(y_test, pred_test, average='macro')


    return {
        'train_acc': train_acc,
        'val_acc': val_acc,
        'test_acc': test_acc,
        'weighted_f1_train': weighted_f1_train,
        'weighted_f1_val': weighted_f1_val,
        'weighted_f1_test': weighted_f1_test,
        'macro_f1_train': macro_f1_train,
        'macro_fq_val': macro_f1_val,
        'macro_f1_test': macro_f1_test
    }

In [ ]:
# Specify the path to the checkpoint
checkpoint_path_scvi = 'checkpoints/scvi_1720603523/'

# Load the scANVI model from the checkpoint
model = scvi.model.SCVI.load(checkpoint_path_scvi, adata=adata_train_val, prefix='sep_knn_class_scvi_cell_type_')

In [20]:
# Specify the path to the checkpoint
checkpoint_path_scanvi = 'checkpoints/scanvi_1720603523/'

# Load the scANVI model from the checkpoint
model_scanvi = scvi.model.SCANVI.load(checkpoint_path_scanvi, adata=adata_train_val, prefix='sep_mlp_class_scanvi_cell_type_')

INFO     File checkpoints/scanvi_1720603523/sep_mlp_class_scanvi_cell_type_model.pt already downloaded             


In [ ]:
# Iterate over separate classifier checkpoints for scanvi
acc_df_scanvi = pd.DataFrame()
for filename in os.listdir('checkpoints/scanvi'):
    if filename.endswith('1720603523.pt'):
        checkpoint_path_class = os.path.join('checkpoints/scanvi', filename)

        # Load the classifier model from the checkpoint
        classifier = Classifier.load(model_scanvi.get_latent_representation(adata), adata.obs['predicted_labels'], checkpoint_path_class)

        # Evaluate the model
        eval_dict = _eval_model(model_scanvi, 'scanvi', classifier=classifier)

        # Print the evaluation results
        class_type = filename.split('_')[2]
        print(f'{class_type} classifier evaluation results:')
        print(eval_dict)

        # Concatenate two pandas dataframes
        acc_df_tmp = pd.DataFrame(eval_dict, index=[0])
        acc_df_tmp['classifier'] = class_type
        acc_df_tmp['model'] = 'scanvi'
        acc_df_tmp['file_name'] = filename

        acc_df_scanvi = pd.concat([acc_df_scanvi, acc_df_tmp])

In [ ]:
# Iterate over separate classifier checkpoints for scvi
acc_df_scvi = pd.DataFrame()
for filename in os.listdir('checkpoints/scvi'):
    if filename.endswith('.pt'):
        checkpoint_path_class = os.path.join('checkpoints/scvi', filename)

        # Load the classifier model from the checkpoint
        classifier = Classifier.load(model.get_latent_representation(adata), adata.obs['predicted_labels'], checkpoint_path_class)

        # Evaluate the model
        eval_dict = _eval_model(model, 'scvi', classifier=classifier)

        # Print the evaluation results
        class_type = filename.split('_')[2]
        print(f'{class_type} classifier evaluation results:')
        print(eval_dict)

        # Concatenate two pandas dataframes
        acc_df_tmp = pd.DataFrame(eval_dict, index=[0])
        acc_df_tmp['classifier'] = class_type
        acc_df_tmp['model'] = 'scvi'
        acc_df_tmp['file_name'] = filename

        acc_df_scvi = pd.concat([acc_df_scvi, acc_df_tmp])

In [7]:
# Results for built-in classifier in scANVI
# Iterate over classifier checkpoints
checkpoint_dir_path = 'outputs/transfer_labeling/scanvi'
acc_df_scanvi = pd.DataFrame()
for filename in os.listdir(checkpoint_dir_path):
    if filename.startswith('scanvi'):
        checkpoint_path_class = os.path.join(checkpoint_dir_path, filename)

        df_tmp = pd.read_csv(checkpoint_path_class)
        df_tmp['file_name'] = filename
        df_tmp['classifier'] = filename.split('_')[1]

        acc_df_scanvi = pd.concat([acc_df_scanvi, df_tmp])
        # condition, predicted_labels, predicted_labels, condition

# could extract this from ['config']['data']['label_key']
acc_df_scanvi['label_key'] = ['condition', 'predicted_labels', 'predicted_labels', 'condition']

In [ ]:
# Concatenate all results for separate classifier checkpoints (see concatenated_df above)
concatenated_df['model'] = 'invae'

acc_df_invae = concatenated_df[['classifier',
 'file_name',
 'macro_f1_test',
 'macro_f1_train',
 'macro_fq_val',
 'model',
 'test_acc',
 'train_acc',
 'val_acc',
 'weighted_f1_test',
 'weighted_f1_train',
 'weighted_f1_val']]

acc_df = pd.concat([acc_df_scvi, acc_df_scanvi, concatenated_df])

In [ ]:
#acc_df.drop(columns=['model_name', 'config', 'batch_key'], inplace=True)
acc_df.drop(columns=['batch'], inplace=True)

acc_df.reset_index(drop=True, inplace=True)

#acc_df.to_csv('outputs/transfer_labeling/separate_classifier_cell_type.csv', index=False)

In [89]:
acc_df = pd.read_csv('outputs/transfer_labeling/separate_classifier_cell_type.csv')

In [ ]:
df_invae_tmp = acc_df.iloc[[0,1,2]]

In [90]:
acc_df['model'] = acc_df['model'] + '_' + acc_df['batch_key']

In [ ]:
# Load all single checkpoints for disease classification (manually otherwise need to check 'label_key' in config)
df_invae_cond_mlp = pd.read_csv('outputs/transfer_labeling/invae/sep_mlp_class_invae_cell_type_1720626976.csv')
df_invae_cond_mlp['classifier'] = 'mlp'

df_invae_cond_linear = pd.read_csv('outputs/transfer_labeling/invae/sep_lin_class_invae_cell_type_1720626976.csv')
df_invae_cond_linear['classifier'] = 'linear'

df_invae_cond_knn = pd.read_csv('outputs/transfer_labeling/invae/sep_knn_class_invae_cell_type_1720626976.csv')
df_invae_cond_knn['classifier'] = 'knn'

df_invae_cond = pd.concat([df_invae_cond_mlp, df_invae_cond_linear, df_invae_cond_knn])

# scanvi disease classification results and cell type
df_scanvi_tmp_cond = acc_df_scanvi[acc_df_scanvi['label_key'] == 'condition']
df_scanvi_tmp = acc_df_scanvi[acc_df_scanvi['label_key'] == 'predicted_labels']

# scpoli disease and cell type classification results
df_scpoli_tmp_cond = pd.read_csv('outputs/transfer_labeling/scpoli_disease.csv')
df_scpoli_tmp_cond['file_name'] = 'scpoli_disease.csv'

df_scpoli_tmp = pd.read_csv('outputs/transfer_labeling/scpoli_cell_type.csv')
df_scpoli_tmp['file_name'] = 'scpoli_cell_type.csv'

In [ ]:
def rename_classifier(x):
    if x == 'lin':
        return 'linear'
    elif x == 'mlp':
        return 'MLP'
    elif x == 'knn':
        return 'kNN'
    
acc_df['classifier_nice'] = acc_df['classifier'].(lambda x: rename_classifier(x))

In [127]:
# Merge them
best_cell_type_df = pd.concat([df_invae_tmp, df_scanvi_tmp, df_scpoli_tmp])
best_cond_df = pd.concat([df_invae_cond, df_scanvi_tmp_cond, df_scpoli_tmp_cond])

# Reset index for selecting rows for plotting
best_cond_df.reset_index(drop=True, inplace=True)
best_cell_type_df.reset_index(drop=True, inplace=True)

# select best models
#best_cond_df = best_cond_df.iloc[[2, 3, 5]]
#best_cell_type_df = best_cell_type_df.iloc[[0,1,2]] #[0, 2, 3]

In [117]:
best_cell_type_df.to_csv('outputs/transfer_labeling/built_in_cell_type_classification.csv', index=False)

In [129]:
best_cond_df.to_csv('outputs/transfer_labeling/built_in_disease_classification.csv', index=False)

In [112]:
best_cell_type_df.loc[[0,1,2], 'model'] = 'invae'
best_cell_type_df.loc[[3,4], 'model'] = 'scanvi'
best_cell_type_df.at[5, 'model'] = 'scpoli'

#best_cell_type_df.at[2, 'classifier'] = 'mlp'

#best_cell_type_df['model'] = best_cell_type_df['model'] + '_' + best_cell_type_df['classifier']

In [114]:
def rename_classifier(x):
    if x == 'lin':
        return 'linear'
    elif x == 'mlp':
        return 'MLP'
    elif x == 'knn':
        return 'kNN'
    else:
        return x
    
best_cell_type_df['classifier_nice'] = best_cell_type_df['classifier'].apply(lambda x: rename_classifier(x))

In [115]:
def rename_model(x):
    if x == 'invae':
        return 'inVAE'
    elif x == 'scanvi':
        return 'scANVI'
    elif x == 'scpoli':
        return 'scPoli'
    else:
        return x
#best_cond_df['model_nice'] = best_cond_df['model_name'].apply(lambda x: rename_model(x))    
best_cell_type_df['model_nice'] = best_cell_type_df['model'].apply(lambda x: rename_model(x))

In [ ]:
# Get unique models from the dataframe
models = acc_df['model'].unique()

# Create a plot for each model
for model in models:
    # Filter the dataframe for the current model
    acc_df_model = acc_df[acc_df['model'] == model]
    
    # Melt the dataframe for easier plotting
    df_melted_model = acc_df_model.melt(id_vars=['classifier_nice'], value_vars=[
        'train_acc', 'val_acc', 'test_acc', 'weighted_f1_train', 
        'weighted_f1_val', 'weighted_f1_test', 'macro_f1_train', 
        'macro_f1_val', 'macro_f1_test'], var_name='metric', value_name='value')
    
    # Set the figure size
    plt.figure(figsize=(15, 8))
    
    # Create a bar plot
    sns.barplot(data=df_melted_model, x='metric', y='value', hue='classifier_nice', dodge=True)
    
    # Customize the plot
    plt.title(f'Performance Model: {model}')
    plt.xlabel('Metric')
    plt.ylabel('Value')
    plt.ylim(0, 1)
    plt.legend(title='Classifier', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Rotate x labels for better readability
    plt.xticks(rotation=45)
    
    # Display the plot
    plt.tight_layout()
    plt.show()


In [ ]:
acc_df = best_cond_df

# Filter the dataframe for the current model
acc_df_model = acc_df

# Melt the dataframe for easier plotting
#df_melted_model = acc_df_model.melt(id_vars=['model'], value_vars=[
#    'train_acc', 'val_acc', 'test_acc', 'weighted_f1_train', 
#    'weighted_f1_val', 'weighted_f1_test', 'macro_f1_train', 
#    'macro_f1_val', 'macro_f1_test'], var_name='metric', value_name='value')
df_melted_model = acc_df_model.melt(id_vars=['model_nice'], value_vars=[
    'test_acc', 'weighted_f1_test', 'macro_f1_test'], var_name='metric', value_name='value')

# Set the figure size
plt.figure(figsize=(15, 8))

# Create a bar plot
sns.barplot(data=df_melted_model, x='metric', y='value', hue='model_nice', dodge=True)

# Customize the plot
plt.title(f'Disease (control or DCM) classification')
plt.xlabel('Metric')
plt.ylabel('Value')
plt.ylim(0, 1)
plt.legend(title='Classifier', bbox_to_anchor=(1.05, 1), loc='upper left')

# Rotate x labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.tight_layout()
plt.show()


# Classification on unseen disease

In [6]:
adata_full = sc.read_h5ad('data/heart_filtered.h5ad')

/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [7]:
adata_full.obs['publication'].value_counts()

Chaffin22     137109
Reichart22    132767
Kanemaru22     53521
Name: publication, dtype: int64

In [4]:
hcm_adata = sc.read_h5ad('data/heart_hcm.h5ad')

/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/ferdinand.kapl/miniconda3/envs/project-inpossible/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
hcm_adata

AnnData object with n_obs × n_vars = 52270 × 4000
    obs: 'donor', 'sex', 'age', 'condition', 'publication', 'kit_10x', 'cell_or_nuclei', 'batch_key', 'cell_type', 'cell_state', 'predicted_labels', 'scrublet_score', 'pred_doublets_per_batch_key', 'pred_scr_thresh_0.2', 'partition'
    var: 'index', 'gene_id', 'gene_name', 'gene_biotype', 'gene_seq_start', 'gene_seq_end', 'seq_name', 'seq_strand', 'seq_coord_system', 'description', 'gene_id_version', 'canonical_transcript', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'age_zero_ten_colors', 'cell_type_colors', 'hvg', 'log1p', 'pred_doublets_per_batch_key_colors'
    layers: 'counts'

In [136]:
tmp_path = 'checkpoints/scanvi_linear_1720624615/linear_class_model.pt'

In [55]:
def calculate_classification_metrics(y_true, y_pred):
    acc = np.mean(y_pred == y_true)

    # F1
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')

    # Calculate the macro F1 score
    macro_f1 = f1_score(y_true, y_pred, average='macro')

    return {
        'acc': acc,
        'weighted_f1': weighted_f1,
        'macro_f1': macro_f1
    }

In [8]:
checkpoints_scanvi_built_in_class = ['checkpoints/scanvi_linear_1720624615/linear_class_model.pt', 'checkpoints/scanvi_mlp_1720624639/mlp_class_model.pt']
y_true = hcm_adata.obs['predicted_labels']
scanvi_unseen_disease_df = pd.DataFrame()

for checkpoint in checkpoints_scanvi_built_in_class:
    tmp_dir, tmp_filename = os.path.split(checkpoint)    
    model = scvi.model.SCANVI.load(tmp_dir, adata=adata_train_val, prefix='_'.join(tmp_filename.split('_')[:-1]) + '_')

    # predict on unseen disease
    y_test = model.predict(hcm_adata)

    metrics = calculate_classification_metrics(y_true, y_test)

    metrics_tmp_df = pd.DataFrame(metrics, index=[0])
    metrics_tmp_df['model'] = 'scanvi'
    metrics_tmp_df['classifier'] = tmp_filename.split('_')[0]

    scanvi_unseen_disease_df = pd.concat([scanvi_unseen_disease_df, metrics_tmp_df])

INFO     File checkpoints/scanvi_linear_1720624615/linear_class_model.pt already downloaded                        


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     File checkpoints/scanvi_mlp_1720624639/mlp_class_model.pt already downloaded                              
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [10]:
checkpoints_invae = ['checkpoints/invae/model_sep_knn_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/model_sep_lin_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/model_sep_mlp_class_invae_cell_type_1720543205.pt']
checkpoint_classifiers_invae = ['checkpoints/invae/class_sep_knn_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/class_sep_lin_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/class_sep_mlp_class_invae_cell_type_1720543205.pt']

y_true = hcm_adata.obs['predicted_labels']
invae_unseen_disease_df = pd.DataFrame()

for chckpt_model, chckpt_class in zip(checkpoints_invae, checkpoint_classifiers_invae):
    tmp_dir, tmp_filename = os.path.split(chckpt_model)    
    # Load Model
    model = FinVAE.load_model(save_path=chckpt_model, adata=adata_train_val, device='cuda')

    # Load Classifier
    classifier = Classifier.load(model.get_latent_representation(adata_train), adata_train.obs['predicted_labels'], chckpt_class)
    
    # predict on unseen disease
    y_test = classifier.predict(model.get_latent_representation(hcm_adata))

    metrics = calculate_classification_metrics(y_true, y_test)

    metrics_tmp_df = pd.DataFrame(metrics, index=[0])
    metrics_tmp_df['model'] = 'invae'
    metrics_tmp_df['classifier'] = tmp_filename.split('_')[2]

    invae_unseen_disease_df = pd.concat([invae_unseen_disease_df, metrics_tmp_df])
# extract classifiers from filenames
class_str = [f.split('_')[2] for f in checkpoints_invae]
invae_unseen_disease_df['classifier'] = class_str

In [20]:
# concat all results
unseen_disease_df = pd.concat([scanvi_unseen_disease_df, invae_unseen_disease_df])
unseen_disease_df.reset_index(drop=True, inplace=True)

In [25]:
tmp_df = pd.read_csv('outputs/transfer_labeling/built_in_disease_classification.csv')

In [28]:
tmp_df['model_nice'] = tmp_df['model_name'] + '_' + tmp_df['classifier']

In [ ]:
acc_df_model = tmp_df

# Melt the dataframe for easier plotting
#df_melted_model = acc_df_model.melt(id_vars=['model'], value_vars=[
#    'train_acc', 'val_acc', 'test_acc', 'weighted_f1_train', 
#    'weighted_f1_val', 'weighted_f1_test', 'macro_f1_train', 
#    'macro_f1_val', 'macro_f1_test'], var_name='metric', value_name='value')
df_melted_model = acc_df_model.melt(id_vars=['model_nice'], value_vars=[
    'acc', 'weighted_f1', 'macro_f1'], var_name='metric', value_name='value')

# Set the figure size
plt.figure(figsize=(15, 8))

# Create a bar plot
sns.barplot(data=df_melted_model, x='metric', y='value', hue='model_nice', dodge=True)

# Customize the plot
plt.title(f'Cell type classification on unseen disease from new dataset')
plt.xlabel('Metric')
plt.ylabel('Value')
plt.ylim(0, 1)
plt.legend(title='Classifier', bbox_to_anchor=(1.05, 1), loc='upper left')

# Rotate x labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
acc_df_model = tmp_df

# Melt the dataframe for easier plotting
#df_melted_model = acc_df_model.melt(id_vars=['model'], value_vars=[
#    'train_acc', 'val_acc', 'test_acc', 'weighted_f1_train', 
#    'weighted_f1_val', 'weighted_f1_test', 'macro_f1_train', 
#    'macro_f1_val', 'macro_f1_test'], var_name='metric', value_name='value')
df_melted_model = acc_df_model.melt(id_vars=['model_nice'], value_vars=[
    'test_acc', 'weighted_f1_test', 'macro_f1_test'], var_name='metric', value_name='value')

# Set the figure size
plt.figure(figsize=(15, 8))

# Create a bar plot
sns.barplot(data=df_melted_model, x='metric', y='value', hue='model_nice', dodge=True)

# Customize the plot
plt.title(f'Disease')
plt.xlabel('Metric')
plt.ylabel('Value')
plt.ylim(0, 1)
plt.legend(title='Classifier', bbox_to_anchor=(1.05, 1), loc='upper left')

# Rotate x labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.tight_layout()
plt.show()

## Plot confusion matrices

In [13]:
tmp_df = pd.read_csv('outputs/transfer_labeling/built_in_disease_classification.csv')

In [9]:
adata_train_val = adata_train_val.copy()

In [51]:
# Data and label
plot_adata = adata_test #hcm_adata
label_key = 'condition'
plot_only_subset = True

y_true = plot_adata.obs[label_key]

# Checkpoints

## scANVI
if label_key == 'predicted_labels':
    ### Cell type
    checkpoints_scanvi_built_in_class = ['checkpoints/scanvi_linear_1720624615/linear_class_model.pt', 'checkpoints/scanvi_mlp_1720624639/mlp_class_model.pt']
elif label_key == 'condition':
    ### Disease
    checkpoints_scanvi_built_in_class = ['checkpoints/scanvi_linear_1720622826/linear_class_model.pt', 'checkpoints/scanvi_mlp_1720622830/mlp_class_model.pt']

## inVAE
if label_key == 'predicted_labels':
    ### Cell type
    checkpoints_invae = ['checkpoints/invae/model_sep_knn_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/model_sep_lin_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/model_sep_mlp_class_invae_cell_type_1720543205.pt']
    checkpoint_classifiers_invae = ['checkpoints/invae/class_sep_knn_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/class_sep_lin_class_invae_cell_type_1720543205.pt', 'checkpoints/invae/class_sep_mlp_class_invae_cell_type_1720543205.pt']
elif label_key == 'condition':
    ### Disease
    checkpoints_invae = ['checkpoints/invae/model_sep_knn_class_invae_cell_type_1720626976.pt', 'checkpoints/invae/model_sep_lin_class_invae_cell_type_1720626976.pt', 'checkpoints/invae/model_sep_mlp_class_invae_cell_type_1720626976.pt']
    checkpoint_classifiers_invae = ['checkpoints/invae/class_sep_knn_class_invae_cell_type_1720626976.pt', 'checkpoints/invae/class_sep_lin_class_invae_cell_type_1720626976.pt', 'checkpoints/invae/class_sep_mlp_class_invae_cell_type_1720626976.pt']

# Sorted labels in decreasing order of frequency
unique_labels = y_true.value_counts().sort_values(ascending=False).index.tolist()

# For disease only the cell types where disease is expected to have an effect
if plot_only_subset and label_key == 'condition':
    int_ct = [
        'Fibroblast',
        'Ventricular Cardiomyocyte',
        'Endothelial cell',
        'Myeloid',
        'Lymphatic Endothelial cell',
        'Atrial Cardiomyocyte',
    ]
else:
    int_ct = unique_labels

In [ ]:
scanvi_df = pd.DataFrame()

for checkpoint in checkpoints_scanvi_built_in_class:
    tmp_dir, tmp_filename = os.path.split(checkpoint)    
    model = scvi.model.SCANVI.load(tmp_dir, adata=adata_train_val, prefix='_'.join(tmp_filename.split('_')[:-1]) + '_')

    # predict on unseen disease
    y_pred = model.predict(plot_adata)

    # Subset for evaluation
    y_true_subset = y_true[plot_adata.obs['predicted_labels'].isin(int_ct)]
    y_pred_subset = y_pred[plot_adata.obs['predicted_labels'].isin(int_ct)]

    # Compute the confusion matrix
    cm = confusion_matrix(y_true_subset, y_pred_subset, labels=unique_labels)

    # Plot the confusion matrix using Matplotlib
    fig, ax = plt.subplots()
    ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=ax)
    plt.show()

    # Plot the confusion matrix using Seaborn
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f"Confusion Matrix for scANVI with {tmp_filename.split('_')[0]} classifier {f'for only subset' if plot_only_subset else ''}")
    plt.show()

    metrics = calculate_classification_metrics(y_true_subset, y_pred_subset)

    metrics_tmp_df = pd.DataFrame(metrics, index=[0])
    metrics_tmp_df['model'] = 'scanvi'
    metrics_tmp_df['classifier'] = tmp_filename.split('_')[0]

    scanvi_df = pd.concat([scanvi_df, metrics_tmp_df])


In [ ]:
invae_df = pd.DataFrame()

for chckpt_model, chckpt_class in zip(checkpoints_invae, checkpoint_classifiers_invae):
    tmp_dir, tmp_filename = os.path.split(chckpt_model)    
    # Load Model
    model = FinVAE.load_model(save_path=chckpt_model, adata=adata_train_val, device='cuda')

    # Load Classifier
    classifier = Classifier.load(model.get_latent_representation(adata_train), adata_train.obs[label_key], chckpt_class)
    
    # predict on unseen disease
    y_pred = classifier.predict(model.get_latent_representation(plot_adata))

    # Subset for evaluation
    y_true_subset = y_true[plot_adata.obs['predicted_labels'].isin(int_ct)]
    y_pred_subset = y_pred[plot_adata.obs['predicted_labels'].isin(int_ct)]

    # Compute the confusion matrix
    cm = confusion_matrix(y_true_subset, y_pred_subset, labels=unique_labels)

    # Plot the confusion matrix using Matplotlib
    fig, ax = plt.subplots()
    ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=ax)
    plt.show()

    # Plot the confusion matrix using Seaborn
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=unique_labels, yticklabels=unique_labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f"Confusion Matrix for inVAE with {tmp_filename.split('_')[2]} classifier {f'for only subset' if plot_only_subset else ''}")
    plt.show()

    metrics = calculate_classification_metrics(y_true_subset, y_pred_subset)

    metrics_tmp_df = pd.DataFrame(metrics, index=[0])
    metrics_tmp_df['model'] = 'invae'
    metrics_tmp_df['classifier'] = tmp_filename.split('_')[2]

    invae_df = pd.concat([invae_df, metrics_tmp_df])

In [58]:
# Concat results
int_ct_disase_df = pd.concat([scanvi_df, invae_df])

In [67]:
int_ct_disase_df['model_nice'] = int_ct_disase_df['model'] + '_' + int_ct_disase_df['classifier']

In [ ]:
acc_df_model = int_ct_disase_df

# Melt the dataframe for easier plotting
#df_melted_model = acc_df_model.melt(id_vars=['model'], value_vars=[
#    'train_acc', 'val_acc', 'test_acc', 'weighted_f1_train', 
#    'weighted_f1_val', 'weighted_f1_test', 'macro_f1_train', 
#    'macro_f1_val', 'macro_f1_test'], var_name='metric', value_name='value')
df_melted_model = acc_df_model.melt(id_vars=['model_nice'], value_vars=[
    'acc', 'weighted_f1', 'macro_f1'], var_name='metric', value_name='value')

# Set the figure size
plt.figure(figsize=(15, 8))

# Create a bar plot
sns.barplot(data=df_melted_model, x='metric', y='value', hue='model_nice', dodge=True)

# Customize the plot
plt.title(f'Disease classification on subset of cells')
plt.xlabel('Metric')
plt.ylabel('Value')
plt.ylim(0, 1)
plt.legend(title='Classifier', bbox_to_anchor=(1.05, 1), loc='upper left')

# Rotate x labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.tight_layout()
plt.show()